In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, layers, optimizers, datasets
import numpy as np

### Data Augmentation
Data를 load할 때 다양한 augmentation 적용

In [2]:
(images_train,labels_train), (images_test,labels_test) = datasets.mnist.load_data()
print('----------------train----------------')
print('images:',images_train.shape, images_train.min(), images_train.max())
print('labels:',labels_train.shape)
images_train = images_train.reshape((60000,28,28,1))
images_train = images_train.astype("float32")/255
print('normalized images:',images_train.shape, images_train.min(), images_train.max())

print('----------------test----------------')
print('images:',images_test.shape, images_test.min(), images_test.max())
print('labels:',labels_test.shape)
images_test = images_test.reshape((10000,28,28,1))
images_test = images_test.astype("float32")/255
print('normalized images:',images_test.shape, images_test.min(), images_test.max())

----------------train----------------
images: (60000, 28, 28) 0 255
labels: (60000,)
normalized images: (60000, 28, 28, 1) 0.0 1.0
----------------test----------------
images: (10000, 28, 28) 0 255
labels: (10000,)
normalized images: (10000, 28, 28, 1) 0.0 1.0


In [22]:
set(labels_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [3]:
import matplotlib.pyplot as plt
from PIL import Image

* Augmentation

In [4]:
# 이미지 사이즈 28*28 -> 32*32 / 이미지 Scaling
data_resize_scale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(32, 32),
    layers.experimental.preprocessing.Rescaling(1./255)
])
    
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomContrast(0.5),
    layers.experimental.preprocessing.Rescaling(1./255)
])

In [5]:
input_shape = (28, 28, 1)
num_classes = 10
model = tf.keras.models.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPool2D(strides=(2,2)),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

### Batch Size
Batch Size : 128

In [25]:
128 * 375, 256*188, len(images_train)

(48000, 48128, 60000)

In [6]:
model_train = model.fit(images_train, labels_train,
                    batch_size=128,
                    epochs=3,
                    validation_split=0.2)

Epoch 1/3
375/375 [==============================] - 86s 223ms/step - loss: 2.0600 - acc: 0.2457 - val_loss: 1.5512 - val_acc: 0.4852
Epoch 2/3
375/375 [==============================] - 78s 208ms/step - loss: 1.4886 - acc: 0.4738 - val_loss: 1.3138 - val_acc: 0.5174
Epoch 3/3
375/375 [==============================] - 78s 207ms/step - loss: 1.3347 - acc: 0.5261 - val_loss: 1.2029 - val_acc: 0.5691


Batch Size : 256

In [7]:
model_train = model.fit(images_train, labels_train,
                    batch_size=256,
                    epochs=3,
                    validation_split=0.2)

Epoch 1/3
188/188 [==============================] - 56s 294ms/step - loss: 1.2629 - acc: 0.5550 - val_loss: 1.1737 - val_acc: 0.5829
Epoch 2/3
188/188 [==============================] - 69s 364ms/step - loss: 1.2328 - acc: 0.5671 - val_loss: 1.1401 - val_acc: 0.5980
Epoch 3/3
188/188 [==============================] - 64s 340ms/step - loss: 1.2049 - acc: 0.5789 - val_loss: 1.0929 - val_acc: 0.6181


### 다양한 모델 구조
layer 수, kernel size, stride, padding

In [8]:
input_shape = (28, 28, 1)
num_classes = 10
model = tf.keras.models.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPool2D(strides=(2,2)),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

### Activation

- 각 layer마다 activation=''으로 넣어도 됨
- tf.keras.layers.Activation('')으로 넣어도 됨
- 어느 방식으로 해도 된다

In [9]:
input_shape = (28, 28, 1)
num_classes = 10
model = tf.keras.models.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='sigmoid', input_shape=input_shape), # sigmoid
    tf.keras.layers.MaxPool2D(strides=(2,2)),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.leaky_relu),
#     tf.keras.layers.LeakyReLU(), # leaky relu
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Activation('tanh'), # tanh
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

### Dropout / Batch Normalization

In [10]:
input_shape = (28, 28, 1)
num_classes = 10
model = tf.keras.models.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3,3), padding='same', input_shape=input_shape),
    tf.keras.layers.Dropout(rate=0.2), # Drop Out
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),

    tf.keras.layers.Conv2D(64, (3,3), padding='same'),
    tf.keras.layers.BatchNormalization(), # Batch Normalization
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

### Loss Function

Categorical Cross Entropy
- label이 one-hot 형태이길 기대함

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [12]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])

Sparse Categorical CrossEntropy
- label이 int 형태이길 기대함

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [14]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy, metrics=['acc'])

MSE

In [15]:
model.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [16]:
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['acc'])

MAE

In [17]:
model.compile(optimizer='adam', loss='MAE', metrics=['acc'])

In [18]:
model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['acc'])

### Optimization

In [19]:
# Stochastic Gradient Descent
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# Stochastic Gradient Descent with momentum
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

# Adagrad
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)

# RMSProp
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

# Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)